# Urban Informatics
# Module 08: Mapping and Web Mapping

In [ ]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from cartopy import crs as ccrs

%matplotlib inline

In [ ]:
gdf_world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf_world.head()

In [ ]:
gdf_cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
gdf_cities.head()

## 1. Choropleth mapping

In [ ]:
# map the world countries directly with geopandas
ax = gdf_world.plot()

In [ ]:
# remove antarctica from our geodataframe, then plot again
mask = (gdf_world['name'] != 'Antarctica') & (gdf_world['pop_est'] > 0)
gdf_world = gdf_world[mask]
ax = gdf_world.plot()

In [ ]:
# color countries by population
fig, ax = plt.subplots(figsize=(9, 9))
ax = gdf_world.plot(ax=ax, column='pop_est')

In [ ]:
# create a column to contain a per-capita gdp indicator
gdf_world['gdp_per_cap'] = gdf_world.gdp_md_est / gdf_world.pop_est
fig, ax = plt.subplots(figsize=(12, 12))
ax = gdf_world.plot(ax=ax, column='gdp_per_cap', cmap='inferno_r', edgecolor='k', lw=0.2)

# turn off the axis so it's just a map
ax.axis('off')
plt.show()

In [ ]:
# inspect the highest GDP per capita countries
gdf_world.sort_values(by='gdp_per_cap', ascending=False).head()

In [ ]:
# drop a couple outliers
labels = gdf_world.sort_values(by='gdp_per_cap', ascending=False).iloc[:2].index
gdf_world = gdf_world.drop(labels)

In [ ]:
# map again
fig, ax = plt.subplots(figsize=(12, 12))
ax = gdf_world.plot(ax=ax, column='gdp_per_cap', cmap='inferno_r', edgecolor='k', lw=0.2)
ax.axis('off')
plt.show()

In [ ]:
# map again, with a different classification scheme
fig, ax = plt.subplots(figsize=(12, 12))
ax = gdf_world.plot(ax=ax, column='gdp_per_cap', cmap='inferno_r', scheme='FisherJenks', edgecolor='k', lw=0.2)
ax.axis('off')
plt.show()

In [ ]:
# now it's your turn
# create a subset geodataframe of only african countries, then plot by gdp per capita


## 2. Projecting

In [ ]:
# what CRS are we using?
gdf_world.crs

More info: http://spatialreference.org/ref/epsg/4326/

In [ ]:
# project data to the robinson projection
robinson = '+proj=robin +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs'
gdf_world = gdf_world.to_crs(robinson)
gdf_world.crs

In [ ]:
# map again
fig, ax = plt.subplots(figsize=(15, 15))
ax = gdf_world.plot(ax=ax, column='gdp_per_cap', cmap='inferno_r', edgecolor='w', lw=0.2)
ax.axis('off')
plt.show()

In [ ]:
# now it's your turn
# calculate the population density of each country as a new column, then map the countries by density


## 3. Mapping multiple layers

Plot capital cities on top of countries basemap

In [ ]:
gdf_cities.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# plot the basemap: the country boundaries
ax = gdf_world.plot(ax=ax, color='w', edgecolor='#aaaaaa', lw=1)

# plot the points: the city lat-lngs
ax = gdf_cities.plot(ax=ax, color='r')

ax.axis('off')
plt.show()

Why do we have one single red dot at lat=0 and lng=0?

In [ ]:
gdf_cities.crs == gdf_world.crs

In [ ]:
# gotta project the gdfs so they're in the same CRSs
gdf_cities = gdf_cities.to_crs(gdf_world.crs)
gdf_cities.crs == gdf_world.crs

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# plot the basemap: the country boundaries
ax = gdf_world.plot(ax=ax, color='w', edgecolor='#aaaaaa', lw=1)

# plot the points: the city lat-lngs
ax = gdf_cities.plot(ax=ax, color='none', edgecolor='#003366', lw=2, alpha=0.7)

ax.axis('off')
plt.show()

In [ ]:
# now it's your turn
# map the entire world basemap, but only plot asian cities on top of it


In [ ]:
# join cities to countries, so we know which country each city belongs to
gdf_cities_countries = gpd.sjoin(gdf_cities, gdf_world, how='inner', op='within')

In [ ]:
# our spatial join isn't perfect, because of the low-resolution country boundaries
gdf_cities_countries.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# plot the basemap: the country boundaries
ax = gdf_world.plot(ax=ax, color='#eeeeee', edgecolor='#999999', lw=1)

# plot the points: the city lat-lngs
ax = gdf_cities_countries.plot(ax=ax, column='gdp_per_cap', cmap='inferno_r', edgecolor='k', lw=1, alpha=0.8)

ax.axis('off')
plt.show()

In [ ]:
# now it's your turn
# create a subset geodataframe of only african cities/countries, then plot the countries as a basemap and the cities colored by gdp per capita


## 4. Choosing colors

https://matplotlib.org/users/colormaps.html

Easy rules to (usually) pick a good color map: 

  - if you have data values rising from some baseline to some maximum, use a perceptually uniform sequential color map.
  - if you have data values diverging in both directions from some meaningful center point (e.g., center is zero and values can range positive or negative) then use a diverging color map
  - avoid rainbow/jet color maps

In [ ]:
cmaps = ['viridis',
         'plasma',
         'inferno',
         'YlOrRd',
         'YlGnBu',
         'summer',
         'autumn',
         'bone',
         'RdPu']

fig, axes = plt.subplots(3, 3, figsize=(12, 8), facecolor='#333333')
for cmap, ax in zip(cmaps, axes.flatten()):
    ax = gdf_world.plot(ax=ax, cmap=cmap)
    ax.set_title(cmap, color='w')
    ax.axis('off')

plt.show()

## 5. Cartopy

https://scitools.org.uk/cartopy/docs/latest/crs/projections.html

In [ ]:
# create a cartopy azimuthal equidistant crs object
ae = ccrs.AzimuthalEquidistant()

In [ ]:
# convert it to a proj4 string compatible with geopandas
crs_ae = ae.proj4_init
gdf_world_ae = gdf_world.to_crs(crs_ae)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection':ae})
ax.add_geometries(gdf_world_ae['geometry'], crs=ae)
plt.show()

In [ ]:
# now it's your turn
# look up a new cartopy projection, then use it to map the countries


#### Cartopy directly with geopandas plotting

In [ ]:
# create a cartopy orthographic crs object, then get a string
ortho = ccrs.Orthographic()
crs_ortho = ortho.proj4_init
crs_ortho

In [ ]:
# project then plot
gdf_world_ortho = gdf_world.to_crs(crs_ortho)
ax = gdf_world_ortho.plot()
ax.axis('off')
plt.show()

In [ ]:
# choose your own lat/lon center
crs_ortho_usa = '+ellps=WGS84 +proj=ortho +lon_0=-100 +lat_0=30 +no_defs'
gdf_world_ortho_usa = gdf_world.to_crs(crs_ortho_usa)
ax = gdf_world_ortho_usa.plot()
ax.axis('off')
plt.show()

In [ ]:
# now it's your turn
# plot an orthographic map of world countries colored by gdp per capita, centered on bangkok


## 6. Folium

Folium lets you map your geodataframe as a leaflet (javascript) web map

In [ ]:
# bin the data into quintiles
bins = list(gdf_world['gdp_per_cap'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))

In [ ]:
# create leaflet web map
m = folium.Map(location=(40, 20), zoom_start=4, tiles='cartodbpositron')

In [ ]:
# add data as choropleth
c = folium.Choropleth(gdf_world, data=gdf_world, bins=bins,
                      columns=['name', 'gdp_per_cap'],
                      key_on='feature.properties.name', 
                      highlight=True, fill_color='YlOrRd', 
                      legend_name='GDP Per Capita').add_to(m)

In [ ]:
# add mouseover tooltip to the countries
c.geojson.add_child(folium.features.GeoJsonTooltip(['name', 'gdp_per_cap']))

# save web map to disk
m.save('webmap.html')

In [ ]:
# display the web map inline
m

In [ ]:
# now it's your turn
# create a new leaflet web map via folium, coloring the countries by population


## 7. Web mapping sans Python

Leaflet: https://leafletjs.com/examples.html

Carto: https://carto.com/developers/carto-js/v3/guides/getting-started/

See also: https://go.carto.com/spatial-data-science-carto-python-webinar-recorded